# # Case: Walmart Recruiting - Store Sales Forecasting.

# # Bibliotecas utilizadas.

In [ ]:
# importando todas as bibliotecas necessárias
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import timedelta

from mpl_toolkits import mplot3d


from sklearn.ensemble import RandomForestRegressor
#from lazypredict.Supervised import LazyRegressor
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore") # ignoring annoying warnings

# # 1. Entendendo o problema de Negócio

O Walmart é uma corporação de varejo que opera como uma rede de supermercados. Neste case o Walmart forneceu dados de 45 lojas onde em cada uma delas há 99 departamentos distribuidas no tempo de 2010-02-05 á 2012-11-1. Nos dados diponibilizados  temos também dados das vendas semanais de cada par loja/departamento.Além disso, o Walmart tem uma politica de remarcação de preços promocionais ao londo do ano, principalmente em datas que precedem feriados importantes.

**Diante disso existe um problema de negócio a ser descoberto:** 

**1.** Qual o o impacto dos feriados nas vendas das lojas? 

**2.** É possível estimar as vendas das lojas por semana em datas futuras de 2012–11–02 a 2013–07-26? Se sim,quais seriam esses valores?

**obs:** Para isso, nos dados disponibilizados estão inclusos quatro semanas dos principais feriados. São eles:

**- Super Bowl:** 12-Feb-10, 11-Feb-11, 10-Feb-12, 8-Feb-13

**- Labor Day:** 10-Sep-10, 9-Sep-11, 7-Sep-12, 6-Sep-13

**- Thanksgiving:** 26-Nov-10, 25-Nov-11, 23-Nov-12, 29-Nov-13

**- Christmas:** 31-Dec-10, 30-Dec-11, 28-Dec-12, 27-Dec-13
 
 

# # 2. Qual modelo de machine learning será utilizado para abordar o problema?


Há pelo menos dois caminhos a serem seguidos neste case: 

**a) Séries Temporais:** Uma vez que as vendas estão dispostas ao longo do tempo e temos que prever para datas futuras um possível modelo seria o clássico ARIMA ou o SARIMA (considera períodos sazonais).

**b) Modelos de regressão:** Uma vez que queremos saber a relção das vendas em relação ao feriado,e prever numericamente valores em datas futuras. 
Como no case há questões como pesos diferentes para feriados na semana, tamanho/tipo da loja e muitas outras variáveis optei por uma modelagem de regressão (Random Forest Regressor)

# # 3 . Métrica utilizada

O Walmart estabeleceu como métrica de demsenpenho o Weighted Mean Absolute Error (WMAE), cuja função matemática é mostrada abaixo.

![](http://latex.codecogs.com/gif.latex?%7BWMAE%7D%20%3D%20%5Cfrac%7B1%7D%7B%5Csum%7Bw_i%7D%7D%20%5Csum_%7Bi%3D1%7D%5En%20w_i%20%7C%20y_i%20-%20%5Chat%7By%7D_i%20%7C)

* n é o número de linhas
* \\( \hat{y}_i \\) é a previsão de vendsas.
* \\( y_i \\) são as vendas atuais.
* \\( w_i \\) São os pesos, onde w = 5 se a semana tiver um feriado, caso contrário será 1.

Nosso objetivo é diminuir essa pontuação o máximo possível, além de prever as vendas semanais. Usaremos essa métrica de desempenho nos modelos de machine leanirng abordados. 

#  # 4. Dados disponibilizados

**Foram disponibilizados 5 arquivos com os seguintes campos:**


**a) features.csv —** Este arquivo contém dados adicionais relacionados à loja, departamento e atividade regional para as datas indicadas. Ele contém os seguintes campos:

**Store —** o número da loja

**Date —** a semana

**Temperature —** temperatura média na região

**Fuel Price —** custo do combustível na região

**MarkDown(1–5) —** dados anônimos relacionados a descontos promocionais que o Walmart está executando. Os dados de redução de preços só estão disponíveis após novembro de 2011 e não estão disponíveis para todas as lojas o tempo todo. Qualquer valor ausente é marcado com um NaN.

**CPI -** o índice de preços ao consumidor

**Unemployment –** a taxa de desemprego

**IsHoliday —** Se o feriado cai durante a semana.

----------------------------------------------------------------------------------------------------------------

**b) train.csv** - (Abrange de 2010–02–05 a 2012-11–01)

**Store —** o número da loja

**Dept —** o número do departamento

**Date —** a semana

**weekly_sales—** vendas para determinado departamento em determinada loja

**IsHoliday —** se a semana é uma semana de feriado especial 

________________________________________________________________________________________________________________

**c) stores.csv —** Este arquivo contém informações anônimas sobre as 45 lojas, indicando o tipo e tamanho da loja.
________________________________________________________________________________________________________________

**d) test.csv —** Este arquivo é idêntico ao train.csv, exceto que retemos as vendas semanais. Você deve prever as vendas para cada trio de loja, departamento e data neste arquivo

_______________________________________________________________________________________________________________
**e) sampleSubmission.csv -** Usado para a submissão deste case no kaggle.

# 4.1 Leitura dos dados

In [ ]:
#Leitura dos dados disponibilizados
df_features = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/features.csv.zip')
df_train = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/train.csv.zip')
df_test = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/test.csv.zip')
df_stores = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/stores.csv')
df_sample_submission = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/sampleSubmission.csv.zip')

# 4.2 Overview dos dados importados

# a) features.csv

In [ ]:
# print das principais informações dos dados e das cinco primeiras linhas.
print(f"Shape of features.csv: {df_features.info()}\n")
df_features.head()

**Inferências:**

* Há 12 colunas e 8190 linhas
* A coluna "Date" deveria estar no formato datetime do pandas.
* variáveis categóricas: MarkDown[1 -5], IsHoliday
* De cara já nos deparamos com valores NaN em alguns campos.

In [ ]:
# Mostra alguns dados estatístico básicos
df_features.describe().T

**Inferências:**

* Erro: Valores negativos em alguns campos.

# b) train.csv

In [ ]:
# print das principais informações dos dados e das cinco primeiras linhas.
print(f"Shape of train.csv: {df_train.info()}\n")
df_train.head()

**Inferências:**

* Há 5 colunas e 421570 linhas.
* A coluna "Date" deveria estar no formato datetime do pandas.
* variáveis categóricas: IsHoliday

In [ ]:
# Mostra alguns dados estatístico básicos
df_train.describe()

**Inferências:**

* Erro: Valores negativos nas vendas.

# c) test.csv

In [ ]:
# print das principais informações dos dados e das cinco primeiras linhas.
print(f"Shape of test.csv: {df_test.info()}\n")
df_test.head()

**Inferências:**

* Há 4 colunas e 115064 linhas.
* A coluna "Date" deveria estar no formato datetime do pandas.
* variáveis categóricas: IsHoliday

In [ ]:
# Mostra alguns dados estatístico básicos
df_test.describe()

# d) stores.csv

In [ ]:
# print das principais informações dos dados e das cinco primeiras linhas.
print(f"Shape of stores.csv: {df_stores.info()}\n")
df_stores.head()

**Inferências:**

* Há 3 colunas e 45 linhas.
* A coluna "Date" deveria estar no formato datetime do pandas.
* variáveis categórica: Type

In [ ]:
# Mostra alguns dados estatístico básicos
df_stores.describe()

#  # 5. Tratamento e limpeza dos dados 

# 5.1 Elaboração de dois dataframes: um de treino e um de teste

# a) Dados de treino: merge entres os arquivos features.csv, stores.csv e train.csv

In [ ]:
# Merge de 3 dataframes para gerar um dataframe final de treino aqui intitulado como"df_train_final"
df_train_final = df_train.merge(df_stores,
                                how='inner', 
                                on='Store')\
                         .merge(df_features, 
                                how='inner', 
                                on=['Store', 'Date', 'IsHoliday'])
                        

In [ ]:
# # print das principais informações dos dados e das cinco primeiras linhas.
print(f"Shape of df_train_final.csv: {df_train_final.info()}")
df_train_final.head()

**Inferências:**

* Há 16 colunas e 421570 linhas.
* A coluna "Date" deveria estar no formato datetime do pandas.
* variáveis categórica: Type, IsHoliday
* Valores com NaN em alguns campos

In [ ]:
df_train_final.describe().T

**Observações:**

* Erro: Valores negativos em alguns campos.

# b) Dados de teste: merge entres os arquivos features.csv, stores.csv e test.csv

In [ ]:
# Merge de 3 dataframes para gerar um dataframe final de treino aqui intitulado como"df_train_final"
df_test_final = df_test.merge(df_stores,
                              how='inner',
                              on='Store')\
                       .merge(df_features,
                              how='inner',
                              on=['Store', 'Date', 'IsHoliday'])

In [ ]:
print(f"Shape of df_test_final.csv: {df_test_final.info()}\n")
df_test_final.head()

**Observações:**

* Há 15 colunas e 115064 linhas.
* A coluna "Date" deveria estar no formato datetime do pandas.
* variáveis categórica: Type, IsHoliday
* Valores com NaN em alguns campos

In [ ]:
df_test_final.describe().T

**Observações:**

* Erro: Valores negativos em alguns campos.

# c) Tratar os campos 'Date' e 'Type'

In [ ]:
# transformando o tipo do campo 'Date' para datetime
df_train_final.Date = pd.to_datetime(df_train_final.Date)
df_test_final.Date = pd.to_datetime(df_test_final.Date)

Acima foi transformado a variável 'Date' de object para o tipo datetime aceito pela biblioteca pandas.

In [ ]:
storetype_values = {'A':3, 'B':2, 'C':1}
df_train_final['Type'] = df_train_final.Type.map(storetype_values)
df_test_final['Type'] = df_test_final.Type.map(storetype_values)

Acima a variável categórica 'Type' foi transformada em variável numérica. Onde,

Type: A, B, C  ----> Type: 3, 2, 1

# d) Elaboração de quatro novas colunas a partir do campo 'Date': Year, Month,Week e Day 

Aqui estamos criando quatro novos campos a partir do campo 'Date' para facilitar as análises e modelagens. Assim temos as seguintes novas colunas:

* Year - respectivo ano de uma data.
* Month - respectivo mês de uma data.
* Day - respectivo dia de uma data.
* Week - valor da semana referente a um ano.

**exemplo:** 

    Date: 2012-10-26
    Year: 2012
    Month: 10
    Day: 26
    Week: 43 (Semana 43 do ano de 2012)
    

In [ ]:
# Criação de 4 novos campos: Year, Month,Week e Day 
df_train_final['Year']=df_train_final['Date'].dt.year
df_train_final['Month']=df_train_final['Date'].dt.month
df_train_final['Week']=df_train_final['Date'].dt.week
df_train_final['Day']=df_train_final['Date'].dt.day

df_test_final['Year']=df_test_final['Date'].dt.year
df_test_final['Month']=df_test_final['Date'].dt.month
df_test_final['Week']=df_test_final['Date'].dt.week
df_test_final['Day']=df_test_final['Date'].dt.day


In [ ]:
#verificando se a ação acima foi feita de forma correta
df_train_final.head()

In [ ]:
#verificando se a ação acima foi feita de forma correta
df_test_final.head()

# e) Verificando os valores ausentes

In [ ]:
# para sabar a quantidade de dados totais em cada campo
df_train_final.info()

In [ ]:
df_train_final.isnull().sum()

Consta valores ausentes nos campos 'MarkDown [1-5]'. São valores expressivos em relação a quantidade total de cada um deles. Diante disso, esses valores ausentes serão imputados como 0.

In [ ]:
# Trocando os valores ausentes pelo valor numérico 0
df_train_final.loc[df_train_final.MarkDown1.isnull() ,'MarkDown1']= 0
df_train_final.loc[df_train_final.MarkDown2.isnull() ,'MarkDown2']= 0
df_train_final.loc[df_train_final.MarkDown3.isnull() ,'MarkDown3']= 0
df_train_final.loc[df_train_final.MarkDown4.isnull() ,'MarkDown4']= 0
df_train_final.loc[df_train_final.MarkDown5.isnull() ,'MarkDown5']= 0

In [ ]:
#checando a quant. de valores ausentes após a mudança
df_train_final.isnull().sum()

In [ ]:
# checando se existe valores NaN
df_train_final.isnull().values.any()

# f) Verificando os valores negativos

In [ ]:
# analisando alguns dados estatíticos
df_train_final.describe().T

Os campos Weekly_Sales,Temperature, MarkDown2, MarkDown3 possuem valores negtivos na coluna 'min'. Desses o único que não faz sentido ter um valor mínimo negativo é o Weekly_Sales, afinal não tem como ter vendas semanais negativas, só positivas ou zeradas.

In [ ]:
# checando o numero de valores negativos no campo Weekly_Sales
#Temos 1285 valores negativos nesse campo
df_train_final[df_train_final.Weekly_Sales<0].shape

In [ ]:
# checando o total de valores no campo Weekly_Sales
# Temos 421570 valores
df_train_final['Weekly_Sales'].shape

A porcentagem de valores negativos é muito pequeno em relação ao todo. Portanto, terá baixo impacto na modelagem a remoção desses valores.

In [ ]:
# removendo os valores negativos do campo Weekly_Sales. 
df_train_final = df_train_final[df_train_final.Weekly_Sales >= 0]

In [ ]:
# verificando se a ação acima foi feita
df_train_final[df_train_final.Weekly_Sales<0].shape

Valores negativos do campo Weekly_Sales foram removidos.

# g) Verificando linhas duplicadas.

In [ ]:
# Somando todas as linhas duplicadas
df_train_final.duplicated().sum()

Não há linhas duplicadas.

#  # 6. Análise exploratória dos dados

# a) Impacto das vendas ao longo dos meses

In [ ]:
# plotagem da relação das datas com as vendas semanais
plt.figure(figsize=(15,5))
df_train_final.groupby(['Date'])['Weekly_Sales'].sum().plot()
plt.title('Vendas Semanais vs Datas')
plt.ylabel('Weekly sales ');

### Inferências

1. Analisando o gráfico vemos quatro picos, gerando altos impactos nas vendas. Dois deles é no fim de novembro(feriado de ação de graças) e os outros dois é no més de dezembro (natal). Já o feriado do super bowl(fevereiro) e do Labor Day geram um impacto médio no aumento das vendas.

2. Há uma ligeira queda nas vendas em dezembro de 2011 comparado ao mesmo mês em 2012.

3. Há uma sazonalidade no gráfico de vendas semanais ao longo do período de tempo que pode ser facilmente vista pelo gráfico.

4. As vendas semanais médias estão flutuando em torno de 4,8*10^7

# b) Impacto das vendas ao longo das semanas do ano.

In [ ]:
# plotagem da relação das semanas com as vendas
plt.figure(figsize=(10,10)) # dimensao do tamanho da figura
sns.relplot(x = 'Week',y = 'Weekly_Sales',data = df_train_final, kind='line',aspect = 2)
#plotagem
plt.title("Vendas vs Semanas do ano ")
plt.grid()
plt.show();

**Inferência:**

Há flutuações ligeiras das semanas ao longo do ano. Da semana 45 a 50 temos dois picos de vendas devido aos feriados do Natal e de Dia de Ação de Graças que acontecem por essas semanas.

# c) Impacto dos markdowns[1-5] nas vendas semanais

In [ ]:
# agrupando cada markdown com o campo date e com isso temos a relação entre os dois.
plt.figure(figsize=(15,6));
df_train_final.groupby(['Date'])['MarkDown1'].sum().plot(label='MarkDown1')
df_train_final.groupby(['Date'])['MarkDown2'].sum().plot(label='MarkDown2')
df_train_final.groupby(['Date'])['MarkDown3'].sum().plot(label='MarkDown3')
df_train_final.groupby(['Date'])['MarkDown4'].sum().plot(label='MarkDown4')
df_train_final.groupby(['Date'])['MarkDown5'].sum().plot(label='MarkDown5')

#legendas e plotagem
plt.grid()
plt.title('MarkDowns [1-5] vs Datas')  
plt.ylabel('MarkDown [1-5]')
plt.legend();

**Inferências:**

1. Não há valores de 'markdowns' disponíveis nas datas anteriores a outubro de 2011.

2. Tivemos picos de 'markdowns' no intervelo de novembro de 2011 a fevereiro de 2012. Intervalo este que é onde ocorrem os seguintes feriados: Natal, Ação de Graças e Super Bowl.
3. Ordem do impacto dos picos citados no item 2:
                                               markdowns3 > markdowns2 > markdowns1 > markdowns4 > markdowns5

# d) Impacto nas vendas com/sem feriados durante a semana

In [ ]:
# dimensão do dataframe de treino
df_train_final.shape

In [ ]:
# print das estatíticas das vendas semanais quando tem feriado durante a semana.
print("Holiday vs Weekly_Sales :\n")


print(df_train_final[df_train_final['IsHoliday']==True]['Weekly_Sales'].describe())


print('\n')

# print das estatíticas das vendas semanais quando não há feriado durante a semana.
print("Non-Holiday vs Weekly_Sales:\n")
print(df_train_final[df_train_final['IsHoliday']==False]['Weekly_Sales'].describe())

**Inferências:**

Médias das vendas quando tem feriado na semana: 17092.566

Médias das vendas quando não tem feriado na semana: 15949.958

Ou seja, os feriados durante a semana geram, em média, mais vendas.
Lembrando que na métrica WMAE o feriado na semana terá peso 5 e,numa situação contrário, terá valor 1.

# e) Média das vendas por lojas

In [ ]:
#relacionando a média das vendas semanais e das lojas 
weekly_sales = df_train_final['Weekly_Sales'].groupby(df_train_final['Store']).mean()

plt.figure(figsize=(20,8)) # dimensão da figura
sns.barplot(weekly_sales.index, weekly_sales.values, palette='muted') 

plt.grid()
plt.title('Média das vendas por lojas', fontsize=18)
plt.ylabel('Vendas', fontsize=16)
plt.xlabel('Lojas', fontsize=16)
plt.show()

**Inferências:**

1. N° das lojas que vendem mais: 2,4,6,10,13,14,19,20,27 e 39.
2. Intervalo de vendas das lojas citadas em '1': De 22 a 30 mil vendas


# f) Média das vendas por departamentos

In [ ]:
#relacionando a média das vendas semanais e dos departamentos 
weekly_sales = df_train_final['Weekly_Sales'].groupby(df_train_final['Dept']).mean()
plt.figure(figsize=(20,10))
sns.barplot(weekly_sales.index, weekly_sales.values, palette='deep')
#plotagem
plt.grid()
plt.title('Média das vendas por departamentos', fontsize=20)
plt.ylabel('Vendas', fontsize=20)
plt.xlabel('Departamentos', fontsize=20)
plt.show()

**Inferências:**

1. N° dos departamentos que vendem mais: 2,8,13,38,40,65,72,90,91,92,94,95
2. Intervalo de vendas dos departamentos citadas em '1': De 32 a 75 mil vendas
3. Departamentos que não obtiveram vendas segundo os dados disponibilizados: 15,39,43,45,47,51,53,54,57,61,62,63,64,66,68,69,70,73,75,76,78,84,86,88,89,

# g) Média das vendas por tamanho e do tipo da loja

In [ ]:
plt.figure(figsize=(15,6))
sns.scatterplot(x=df_train_final.Size, y=df_train_final.Weekly_Sales, hue=df_train_final.Type, s=80, palette='bright');
#plotagem
plt.xticks( fontsize=16)
plt.yticks( fontsize=16)
plt.xlabel('Tamanho da loja', fontsize=18)
plt.ylabel('Vendas', fontsize=18);

**Inferências:**

1. Vendas das lojas do tipo '3' > Vendas das lojas do tipo '2' > Vendas das lojas do tipo '1'
2. Lojas menores, até 45 mil [unidade não especificada] , vendem entre 100 a 180 mil [unidade nao especificada]
3. Lojas de tamanho mediano (80 a 44 mil [unidade não especificada]) vendem entre 290 a 430 mil [unidade nao especificada]
4. Lojas maiores(acima de 175 mil [unidade nao especificada])

# h) Matriz de correlação 

In [ ]:
#Dython instalado
!pip install dython

In [ ]:
# Dado um conjunto de dados, o dython encontrará automaticamente quais recursos são categóricos e quais são 
#numéricos e calculará uma medida relevante de associação entre cada recurso , plotando tudo como um mapa de 
#calor de fácil leitura. .

#é necessário ser instalado com esse comando: !pip install dython 
from dython import nominal
nominal.associations(df_train_final,figsize=(20,10),mark_columns=True);

**Inferências:**

1. correlação entre [Weekly sales] e [Dept] = 0.15
2. correlação entre [Weekly sales] e [Store] = 0.09
3. correlação entre [Weekly sales] e [Type] = 0.19
4. correlação entre [Weekly sales] e [size] = 0.24
5. correlação entre [Weekly sales] e [Markdown1] = 0.05
6. correlação entre [Weekly sales] e [Markdown2] = 0.02
7. correlação entre [Weekly sales] e [Markdown3] = 0.04
8. correlação entre [Weekly sales] e [Markdown4] = 0.04
9. correlação entre [Weekly sales] e [Markdown5] = 0.05

Todos os demais campos possuem baixa correlação.

# 7. Modelo de Machine Learnig Random Forest Regressor

Função que calcula o WMAE:

In [ ]:
# função criada para o calculo da métrica WMAE
# Entradas da função: dataframe, vendas atuais, previsão de vendas
def WMAE(dataset, real, predicted):
  
    weights = dataset.IsHoliday.apply(lambda x: 5 if x else 1) # Valor do peso. Se w = 5 tem feriado durante a semana e w = 1 caso contrário.
    return np.round(np.sum(weights*abs(real-predicted))/(np.sum(weights)), 2) #calculo da formula do WMAE

**O modelo escolhido para este projeto é o Random Forest Regressor. Trata-se de um modelo que possui um método ensemble que garante uma robustez maior e utiliza múltiplas árvores de decisão ('n_estimators' do modelo)  cuja a média das saídas de todas as arvores determinarão o valor final previsto. Dentre os motivos citados no inicio desta análise para a escolha desse modelo há também o fato que terá baixa probabilidade sofrer de overfitting.**

**Obs: a parte ruim do uso deste modelo é seu custo computacional que é alto.**


In [ ]:
# função para prever vendas a partir do modelo random forest
def random_forest(n_estimators, max_depth): # Entradas: numero de arvores e a profundidade delas.
    result = [] 
    for estimator in n_estimators:  # O intuito é "testar" numeros de arvores e profundidades diferentes para assim escolher a melhor combinação
        for depth in max_depth:
            wmaes_cv = []       # é aqui onde será quardado o valor de WMAE
            for i in range(1,5):
                print('k:', i, ', n_estimators:', estimator, ', max_depth:', depth) # conforme o loop acontece é printado os valores utilizados e encontrados
                x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.3) # dividi-se os dados em dois conjuntos. 70% para treino e 30% para teste.
                RF = RandomForestRegressor(n_estimators=estimator, max_depth=depth)# "Floresta" criada
                RF.fit(x_train, y_train) # "floresta" treinada
                predicted = RF.predict(x_test) # previsão encontradas a paritr dos 30% de dados de teste
                wmaes_cv.append(WMAE(x_test, y_test, predicted)) # add o valor do WMAE a variavel 'wmaes_cv'
            print('WMAE:', np.mean(wmaes_cv)) # print do valor de wmae encontrado
            result.append({'Max_Depth': depth, 'Estimators': estimator, 'WMAE': np.mean(wmaes_cv)}) # cria uma lista com a profundidade e numero de arvores e o valor wmae associado.
    return pd.DataFrame(result) # retorna um novo dataframe com valores comentados na linha anterior

**Abaixo tem a lista dos campos que serão utilizados no modelo e o valor a ser previsto.**

In [ ]:
X_train = df_train_final[['Store','Dept','IsHoliday','Size','Week','Type','Year']] # lista de campos do dataframe que serão utilizados
Y_train = df_train_final['Weekly_Sales'] # o valor a ser previsto: Vendas semanais

In [ ]:
# conferindo se ação acima foi feita corretamente
X_train.info()

In [ ]:
n_estimators = [56, 58, 60] # valores para o numero de arvores 
max_depth = [25, 27, 30] # valores para a ser utilizado na profundidade da arvore

random_forest(n_estimators, max_depth) # chamando a função random_forest 

Pelo gráfico acima o melhor WMAE é 1535.1875

n_estimator: 56

Max_Depth: 25

In [ ]:

#treinando o modelo
RF = RandomForestRegressor(n_estimators=56, max_depth=25)
RF.fit(X_train, Y_train)

In [ ]:
# campos que serão utilizados no modelo
X_test = df_test_final[['Store', 'Dept', 'IsHoliday', 'Size', 'Week', 'Type', 'Year']]
predict = RF.predict(X_test) # previsão dos valores a partir do dataframe de teste

**Submissão dos resultados no Kaggle**

In [ ]:
df_sample_submission['Weekly_Sales'] = predict

In [ ]:
df_sample_submission

In [ ]:
df_sample_submission.to_csv('submission.csv', index=False)